In [1]:
# import pandas as pd
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt

# %matplotlib inline
# import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)

In [81]:
# adult_train = pd.read_csv('../Datasets/adult_train.csv', index_col=0)
# adult_train = adult_train.replace({' ?':np.nan}).dropna()
# adult_train = adult_train.reset_index(drop=True)
# adult_train.head()

In [82]:
# from pandas.api.types import is_string_dtype
# from pandas.api.types import is_numeric_dtype
# import pysnooper

# # @pysnooper.snoop()
# def flip_attr(data, attr, target, reverse_flip=False):
#     np.random.seed(100)
    
#     attributes = data[attr].unique().tolist()
#     print('chosen "{0}" attribute to flip, change from {1} to {2}'.format(attr, attributes[0], attributes[1]))
#     cat_features = [i for i in data.columns if is_string_dtype(data[i])]
#     num_features = [i for i in data.columns if is_numeric_dtype(data[i])]
    
#     attr_index = data.loc[data[attr] == attributes[0]].index
#     data = data.iloc[attr_index]
#     data[attr] = data[attr].factorize()[0]
#     cat_data = data[cat_features]
#     num_data = data[num_features]
    
#     data_flip = data.copy()
#     data_flip[attr] = data[attr].apply(lambda x : x ^ 1)
#     flip_num_data = data_flip[num_features]
    
#     X = pd.get_dummies(cat_data, drop_first=True).join(num_data, how='inner')
#     y = target
#     flip_X = pd.get_dummies(cat_data, drop_first=True).join(flip_num_data, how='inner')
    
#     rf_model = RandomForestClassifier(random_state=0).fit(X, y[attr_index])
#     prob = np.amax(rf_model.predict_proba(X), axis=1)
#     flip_prob = np.amax(rf_model.predict_proba(flip_X), axis=1)

#     df_data = data_flip.copy()
#     avg_effect = np.mean(flip_prob) - np.mean(prob)
#     df_data['deviation'] = (flip_prob - prob) - avg_effect
    
#     return df_data

In [83]:
# data = adult_train.drop(['native-country', 'target'], axis=1)
# target = adult_train['target']
# flip_attr(data, 'sex', target)

In [84]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import pysubgroup as ps


def read_data(file_path):
    data = pd.read_csv(file_path, index_col=0)
    # simply drop missing values
    data = data.dropna()
    data = data.reset_index(drop=True)
    return data


# def train_model(data, features, target, classifier):
#     train_data = data[[features]]
#     target_data = data[target]
#     model = classifier.fit(train_data.values, target_data.values.ravel())
#     return model
#
# def flip_value(data, attr):
#
#     data[attr] = pd.get_dummies(data[attr], drop_first=True)
#     data[attr] = data[attr].apply(lambda x: x ^ 1)
#     return data

def effect_change(data, features, attr, target, classifier):
    train_data = pd.get_dummies(data[features], drop_first=True)
    target_data = data[target]
    df_attr = pd.get_dummies(data[attr], drop_first=True)
    col_name = df_attr.columns[0]
    t_index = df_attr.loc[df_attr[col_name] == True].index
    f_index = df_attr.loc[df_attr[col_name] == False].index

    flip_data = train_data.copy()
    train_data = train_data.join(df_attr, how='inner')
    flip_attr = df_attr.apply(lambda x: x ^ 1)

    flip_train_data = flip_data.join(flip_attr, how='inner')
    model = classifier.fit(train_data.values, target_data.values.ravel())
    prob = np.amax(model.predict_proba(train_data), axis=1)
    prob_flip = np.amax(model.predict_proba(flip_train_data), axis=1)

    diff = np.mean(prob[t_index]) - np.mean(prob[f_index])
    flip_diff = np.mean(prob_flip[f_index]) - np.mean(prob_flip[t_index])

    avg_effect = flip_diff - diff

    df = data[features].copy()
    df['deviation'] = (prob_flip - prob) - avg_effect
    # print(df.head())
    return df


def numeric_discovery(df):
    target = ps.NumericTarget('deviation')
    search_space = ps.create_nominal_selectors(df, ignore='deviation')
    task = ps.SubgroupDiscoveryTask(df, target, search_space, qf=ps.StandardQFNumeric(1))
    result = ps.BeamSearch().execute(task)

    df_dis = ps.as_df(df, result, statistics_to_show=ps.all_statistics_numeric)
    return df_dis


In [87]:
adult_train = pd.read_csv('../Datasets/adult_train.csv', index_col=0)
adult_train = adult_train.replace({' ?':np.nan}).dropna()
adult_train = adult_train.reset_index(drop=True)

features = ['education', 'occupation', 'race', 'marital-status']
target = 'target'
attr = 'sex'
classifier = RandomForestClassifier(random_state=0)
df = effect_change(adult_train, features, attr, target, classifier)
numeric_discovery(df)

,quality,subgroup,size_sg,size_dataset,mean_sg,mean_dataset,std_sg,std_dataset,median_sg,median_dataset,max_sg,max_dataset,min_sg,min_dataset,mean_lift,median_lift
0,183.180723,marital-status= Married-civ-spouse,14065.0,30162.0,-0.172169,-0.185193,0.127375,0.11874,-0.171459,-0.187234,0.286271,0.312766,-0.687234,-0.687234,0.929674,0.915746
1,172.877812,race= White AND marital-status= Married-civ-sp...,12656.0,30162.0,-0.171534,-0.185193,0.121689,0.11874,-0.171459,-0.187234,0.250481,0.312766,-0.624949,-0.687234,0.926241,0.915746
2,94.716892,race= White AND education= Assoc-voc AND marit...,611.0,30162.0,-0.030174,-0.185193,0.143801,0.11874,0.033366,-0.187234,0.169269,0.312766,-0.543737,-0.687234,0.162932,-0.178207
3,94.303330,education= Assoc-voc AND marital-status= Marri...,660.0,30162.0,-0.042309,-0.185193,0.155198,0.11874,0.033366,-0.187234,0.171100,0.312766,-0.555091,-0.687234,0.228461,-0.178207
4,91.474815,race= White AND occupation= Craft-repair,3644.0,30162.0,-0.160090,-0.185193,0.097979,0.11874,-0.168100,-0.187234,0.271508,0.312766,-0.677234,-0.687234,0.864451,0.897808
5,83.012614,occupation= Craft-repair,4030.0,30162.0,-0.164595,-0.185193,0.100507,0.11874,-0.187234,-0.187234,0.271508,0.312766,-0.677234,-0.687234,0.888772,1.000000
6,78.321463,race= White AND education= Assoc-voc,1142.0,30162.0,-0.116611,-0.185193,0.183168,0.11874,-0.122174,-0.187234,0.210623,0.312766,-0.585091,-0.687234,0.629670,0.652522
7,77.479939,race= White AND occupation= Farming-fishing,910.0,30162.0,-0.100051,-0.185193,0.104793,0.11874,-0.118100,-0.187234,0.173719,0.312766,-0.548186,-0.687234,0.540249,0.630760
8,75.780733,occupation= Farming-fishing,989.0,30162.0,-0.108570,-0.185193,0.105681,0.11874,-0.125962,-0.187234,0.173719,0.312766,-0.548186,-0.687234,0.586251,0.672750
9,75.475822,occupation= Prof-specialty AND marital-status=...,2066.0,30162.0,-0.148661,-0.185193,0.088634,0.11874,-0.138323,-0.187234,0.210623,0.312766,-0.585091,-0.687234,0.802734,0.738774
